In [9]:
import vectorbt as vbt
import pandas as pd
import numpy as np
import yaml
from typing import Optional

In [10]:
with open('config.yaml', 'r') as fp:
    config = yaml.safe_load(fp)

"""1. Import Config"""
DATA_FP = config['DATA_FP']
DATA_TF = config['DATA_TF']
FEES = config['FEES']
SLIPPAGE = config['SLIPPAGE']
SIZE = config['SIZE']
SIZE_TYPE = config['SIZE_TYPE']
INIT_BALANCE = config['INIT_BALANCE']
BACKTESTING_DATES_START = config['BACKTESTING_DATES']['START']
BACKTESTING_DATES_END = config['BACKTESTING_DATES']['END']
WICK_RATIO = config['WICK_RATIO']
OP_WICK_RATIO = config['OP_WICK_RATIO']
RR = config['RR']

In [11]:
"""2. Read and process data."""
df_1hour = pd.read_csv(DATA_FP)
df_1hour = df_1hour.set_index('Time')
df_1hour.index = pd.to_datetime(df_1hour.index)
df_1hour = df_1hour[BACKTESTING_DATES_START: BACKTESTING_DATES_END]

In [12]:
"""3. Mark signals, SL, TP. """
df_1hour['Direction'] = -1
df_1hour.loc[df_1hour['Close'] > df_1hour['Open'], 'Direction'] = 1
df_1hour.loc[df_1hour['Direction'] == 1, 'UWL'] = df_1hour['High'] - df_1hour['Close']
df_1hour.loc[df_1hour['Direction'] == -1, 'UWL'] = df_1hour['High'] - df_1hour['Open']
df_1hour.loc[df_1hour['Direction'] == 1, 'LWL'] = df_1hour['Open'] - df_1hour['Low']
df_1hour.loc[df_1hour['Direction'] == -1, 'LWL'] = df_1hour['Close'] - df_1hour['Low']
df_1hour['Body'] = (df_1hour['Close'] - df_1hour['Open']).abs()

bullish_entry_mask = (
    (df_1hour['LWL'] >= WICK_RATIO * df_1hour['Body']) &
    (df_1hour['Direction'] == 1) &
    (df_1hour['UWL'] * OP_WICK_RATIO <= df_1hour['LWL'])
)

bearish_entry_mask = (
    (df_1hour['UWL'] >= WICK_RATIO * df_1hour['Body']) &
    (df_1hour['Direction'] == -1) &
    (df_1hour['LWL'] * OP_WICK_RATIO <= df_1hour['UWL'])
)

df_1hour['Bullish Entry'] = bullish_entry_mask
df_1hour['Bearish Entry'] = bearish_entry_mask

df_1hour = df_1hour.drop(columns=['UWL', 'LWL', 'Body', 'Direction'])
df_1hour.loc[df_1hour['Bullish Entry'] == True, 'SL'] = df_1hour['Low']
df_1hour.loc[df_1hour['Bearish Entry'] == True, 'SL'] = df_1hour['High']

df_1hour.loc[df_1hour['Bullish Entry'] == True, 'TP'] = df_1hour['Close'] + (df_1hour['Close'] - df_1hour['Low']) * RR
df_1hour.loc[df_1hour['Bearish Entry'] == True, 'TP'] = df_1hour['Close'] - (df_1hour['High'] - df_1hour['Close']) * RR

In [13]:
"""4. Main Loop"""

index_arr_1hour = df_1hour.index.to_numpy()
bullish_entry_arr_1hour = df_1hour['Bullish Entry'].to_numpy()
bearish_entry_arr_1hour = df_1hour['Bearish Entry'].to_numpy()
sl_arr_1hour = df_1hour['SL'].to_numpy()
tp_arr_1hour = df_1hour['TP'].to_numpy()
high_arr = df_1hour['High'].to_numpy()
low_arr = df_1hour['Low'].to_numpy()
close_arr = df_1hour['Close'].to_numpy()

price_arr = np.full(len(index_arr_1hour), np.nan)

bullish_entries_arr = np.full(len(index_arr_1hour), False)
bearish_entries_arr = np.full(len(index_arr_1hour), False)

bullish_exit_arr = np.full(len(index_arr_1hour), False)
bearish_exit_arr = np.full(len(index_arr_1hour), False)

opened_trade_direction = None # bullish/bearish/None
current_sl = None; current_tp = None # float/None

for i in range(len(index_arr_1hour)):
    if opened_trade_direction is None: # if trade is not opened, check conditions to open
        bullish_signal = bullish_entry_arr_1hour[i]
        bearish_signal = bearish_entry_arr_1hour[i]
        
        if not bullish_signal and not bearish_signal: # no entry
            continue

        opened_trade_direction = 'bullish' if bullish_signal else 'bearish' # bullish/bearish entry
        bullish_entries_arr[i] = True if bullish_signal else False
        bearish_entries_arr[i] = True if bearish_signal else False
        price_arr[i] = close_arr[i]
        current_sl, current_tp = sl_arr_1hour[i], tp_arr_1hour[i]

        continue

    elif opened_trade_direction == 'bullish':
        close_ = False
        closing_price = None

        if low_arr[i] <= current_sl: # sl hit
            close_ = True
            closing_price = current_sl

        elif high_arr[i] >= current_tp: # tp hit
            close_ = True
            closing_price = current_tp

        
        if close_:
            price_arr[i] = closing_price
            bullish_exit_arr[i], opened_trade_direction, current_sl, current_tp = True, None, None, None
            

    elif opened_trade_direction == 'bearish':
        close_ = False
        closing_price = None

        if high_arr[i] >= current_sl: # sl hit
            close_ = True
            closing_price = current_sl

        elif low_arr[i] <= current_tp: # tp hit
            close_ = True
            closing_price = current_tp

        
        if close_:
            price_arr[i] = closing_price
            bearish_exit_arr[i], opened_trade_direction, current_sl, current_tp = True, None, None, None

In [14]:
"""5. Backtest with Vectorbt"""
pf = vbt.Portfolio.from_signals(
    close=df_1hour['Close'],
    price=price_arr,
    entries=bullish_entries_arr,
    exits=bullish_exit_arr,
    short_entries=bearish_entries_arr,
    short_exits=bearish_exit_arr,
    size=SIZE,
    size_type=SIZE_TYPE,
    fees=FEES,
    slippage=SLIPPAGE,
    init_cash=INIT_BALANCE,
    freq=DATA_TF
)


In [16]:
pf.stats()

Start                         2023-01-02 23:00:00+00:00
End                           2025-12-01 23:00:00+00:00
Period                                719 days 12:00:00
Start Value                                     10000.0
End Value                                  10529.633734
Total Return [%]                               5.296337
Benchmark Return [%]                         131.081934
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              13.513342
Max Drawdown Duration                 363 days 19:00:00
Total Trades                                       1371
Total Closed Trades                                1370
Total Open Trades                                     1
Open Trade PnL                                      0.0
Win Rate [%]                                  20.875912
Best Trade [%]                                 3.326218
Worst Trade [%]                               -1